In [2]:
# ===============================
# StudyMate AI – Business Agent
# ===============================

import json
import time
import re
import os
from dotenv import load_dotenv
from pypdf import PdfReader
import gradio as gr
import google.generativeai as genai

# ===============================
# Load API Key
# ===============================
load_dotenv()
genai.configure(api_key=os.getenv("GEMINI_API_KEY"))

model = genai.GenerativeModel("gemini-pro")

# ===============================
# Read business files
# ===============================
def read_pdf(path):
    reader = PdfReader(path)
    text = ""
    for page in reader.pages:
        text += page.extract_text() or ""
    return text

summary_text = open("me/business_summary.txt", "r", encoding="utf-8").read()
pdf_text = read_pdf("me/about_business.pdf")

business_context = summary_text + "\n\n" + pdf_text

# ===============================
# Tool functions (REQUIRED)
# ===============================
def record_customer_interest(email, name, message):
    lead = {
        "time": time.time(),
        "email": email,
        "name": name,
        "message": message
    }
    with open("leads.jsonl", "a", encoding="utf-8") as f:
        f.write(json.dumps(lead) + "\n")
    return "Lead saved."

def record_feedback(question):
    fb = {
        "time": time.time(),
        "question": question
    }
    with open("feedback.jsonl", "a", encoding="utf-8") as f:
        f.write(json.dumps(fb) + "\n")
    return "Feedback saved."

# ===============================
# Helper
# ===============================
def extract_email(text):
    m = re.search(r"[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Za-z]{2,}", text)
    return m.group(0) if m else None

state = {"pending_email": None}

# ===============================
# Chatbot Logic
# ===============================
def chat_bot(message, history):
    email = extract_email(message)

    if email:
        state["pending_email"] = email
        return "Thanks! Please tell me your name and what you need."

    if state["pending_email"]:
        name = message.split()[0]
        record_customer_interest(
            email=state["pending_email"],
            name=name,
            message=message
        )
        state["pending_email"] = None
        return "Perfect ✅ Your interest is recorded."

    prompt = f"""
You are StudyMate AI, a business chatbot.

Business info:
{business_context}

User question:
{message}
"""

    try:
        response = model.generate_content(prompt)
        return response.text
    except:
        record_feedback(message)
        return "I couldn't answer that, but I saved it as feedback."

# ===============================
# Gradio Interface (REQUIRED)
# ===============================
gr.ChatInterface(
    chat_bot,
    title="StudyMate AI – Business Agent"
).launch(share=True)


* Running on local URL:  http://127.0.0.1:7861

Could not create share link. Please check your internet connection or our status page: https://status.gradio.app.
